# Overview

We are using this Delivery Hero Recommendation Dataset. 
https://github.com/deliveryhero/dh-reco-dataset

The data includes Foodpanda data from Singapore and Taiwan, and foodora data from Sweden.*

The goal is to analyse the data and use it to validate and support our proposed solutions.

*https://www.deliveryhero.com/wp-content/uploads/2024/03/2023-EXTERNAL-DH-GHG-Accounting-Methodology_FINAL_Branded.pdf



# Data Cleaning

Set `data_cleaning` to `True` to enable the data cleaning block

In [4]:
data_cleaning = False

## Converting geohash to lat/lon
The dataset included geohashes for each order. To visualise it using kepler.gl, converting this to latitude/longitude was required.

In [22]:
import pandas as pd

if data_cleaning:
	path = "orders_sg.csv"
	df = pd.read_csv(path)
	df.head()

In [24]:
import geohash
import os

def convert_geohash(file):
    df = pd.read_csv(file)
    try:
        df['geohash'].values.any()
    except KeyError:
        print("No geohash in file")
        return

    dir = os.getcwd()
    print(f"Processing {file}")
    name, ext = os.path.splitext(file)
    output_file = f'{dir}/{name}_clean.csv'  # Path to your output CSV file
    # Create empty lists to store the latitudes and longitudes
    latitudes = []
    longitudes = []

    # Iterate over each geohash in the DataFrame, decode it, and append the values
    for geohash_str in df['geohash']:
        lat, lon = geohash.decode(geohash_str)  # Decode the geohash
        latitudes.append(lat)
        longitudes.append(lon)

    # Add the latitude and longitude columns to the DataFrame
    df['latitude'] = latitudes
    df['longitude'] = longitudes

    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_file, index=False)

In [25]:
def batch_convert_files():
	files = ["orders_sg_test.csv",
			"orders_sg_train.csv",
			"orders_sg.csv",
			"products_sg.csv",
			"vendors_sg.csv"]
	for file in files:
		convert_geohash(file)


In [8]:
if data_cleaning:
	batch_convert_files()

# Getting orders by time frame (orders within a similar area that can be combined)

In [1]:
# Load Data Using Google Drive URL
import pandas as pd
import numpy as np

def get_gd_csv(gd_id):
	url = f"https://drive.usercontent.google.com/download?id={gd_id}&export=download&authuser=0&confirm=t&uuid=7e92ecad-4c98-448f-8c21-cada2eefe64c&at=AO7h07fO3-BbnWsezAxoi78L-h_0%3A1727109215042"
	df = pd.read_csv(url)
	return df

vendors_gd = '1FPYHV73PfNmC3p8AZEYeyoatJy9XUbAE'
orders_gd = '1ag0k8lzUWQ8J-CopiR4fjN_qv60ErLll'
products_gd = '13V5LUuQB8-VTZX45Eh_UynuQ0ZatAebf'

vendors = get_gd_csv(vendors_gd)
orders = get_gd_csv(orders_gd)
products = get_gd_csv(products_gd)

# Rename columns
vendors.rename(columns={'geohash': 'vendor_geohash', 'latitude': 'vendor_latitude', 'longitude': 'vendor_longitude'}, inplace=True)
orders.rename(columns={'geohash': 'customer_geohash', 'latitude': 'customer_latitude', 'longitude': 'customer_longitude'}, inplace=True)

# Merge orders with vendors and products to get a complete view
merged_data = orders.merge(vendors, on="vendor_id").merge(products, on=["vendor_id", "product_id"])

# Apply the segmentation rule
def segment_delivery(row):
    if 'grocery' in row['name'].lower() or 'combo' in row['name'].lower() or row['primary_cuisine'] == 'grocery':
        return 'Grocery Delivery'
    else:
        return 'Food Delivery'

# Create a new column in the merged dataframe for the segmentation
merged_data['delivery_type'] = merged_data.apply(segment_delivery, axis=1)

# Output the segmented data
food_delivery = merged_data[merged_data['delivery_type'] == 'Food Delivery']
groceries_delivery = merged_data[merged_data['delivery_type'] == 'Grocery Delivery']

# Create a new dataframe to work with
new = food_delivery.copy()

# Change the format of some columns for easier processing
new["order_day"] = new["order_day"].str.replace(' days', '', regex=False)
new['order_day'] = new['order_day'].astype(int)
new["order_time"] = new["order_time"].str.replace(':', '', regex=False)
new['order_time'] = new['order_time'].astype(int)

# Add a column to combine time info from order_time and order_day
def hhmmss_to_seconds(hhmmss):
    hours = hhmmss // 10000
    minutes = (hhmmss % 10000) // 100
    seconds = hhmmss % 100
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return total_seconds

new['time_delta'] = new['order_time'].apply(hhmmss_to_seconds) + (24 * 3600 * new['order_day'])

# Group time info by 20 min ranges
min_value = new['time_delta'].min()
max_value = new['time_delta'].max()
bins = range(min_value, max_value + (60 * 20), (60 * 20))

# Create the bins using pd.cut()
new['time_range'] = pd.cut(new['time_delta'], bins=bins, right=False)

# Sort by the number of occurences in each time frame
new.groupby('time_range').size().sort_values(ascending=False)

KeyboardInterrupt: 

In [67]:

# store info from a specific time range
peak = new[(new['time_delta'] >= 2485215) & (new['time_delta'] <= 2486415)]

# filter to get deliveries to different areas
filtered = peak[peak['vendor_geohash'] != peak['customer_geohash']]

# sort by the number of unique order ids from the same restaurant to a specific area
filtered.groupby(['chain_id','vendor_geohash', 'customer_geohash'])['order_id'].nunique().sort_values(ascending=False)

chain_id  vendor_geohash  customer_geohash
d7b22699  w23bh           w21zu               4
e30f982a  w21zg           w21zu               2
24975bf7  w23bh           w21zu               2
9ff973ca  w23bh           w21zu               2
5fec0adf  w21z7           w21z6               2
                                             ..
57e29f5c  w21zu           w21zg               1
57a7f0fe  w21z7           w21z6               1
57a40432  w21z7           w21zk               1
578b65f3  w21z7           w21ze               1
5daed5ee  w23bh           w21zu               1
Name: order_id, Length: 336, dtype: int64

In [69]:

# show a specific result
temp = peak[(peak["chain_id"] == "d7b22699") & (peak["vendor_geohash"] == "w23bh") & (peak["customer_geohash"] == "w21zu")]
temp


,Unnamed: 0_x,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day,customer_latitude,...,vendor_geohash,primary_cuisine,vendor_latitude,vendor_longitude,Unnamed: 0,name,unit_price,delivery_type,time_delta,time_range
357810,418424,1a6d7aef2c,w21zu,203668,2b6fa394,7e60fe5af2d1,6,182206,28,1.384277,...,w23bh,chinese,1.428223,103.908691,937813,Soup Of The Day 每日老火例汤,0.0036,Food Delivery,2485326,"[2485215, 2486415)"
357811,418425,1a6d7aef2c,w21zu,203668,2b6fa394,614ff9a3d68f,6,182206,28,1.384277,...,w23bh,chinese,1.428223,103.908691,732277,Wintermelon with Conpoy 原盅冬瓜炖瑶柱,0.0044,Food Delivery,2485326,"[2485215, 2486415)"
1937493,2270408,5d0e0a6ba9,w21zu,1118609,2b6fa394,880aa84b75be,6,182354,28,1.384277,...,w23bh,chinese,1.428223,103.908691,676145,Steamed Fish Slices With Shredded Pork 古法蒸生鱼片,0.0068,Food Delivery,2485434,"[2485215, 2486415)"
2956894,3468959,e176e909ae,w21zu,1729632,2b6fa394,4b72eed497a7,6,182223,28,1.384277,...,w23bh,chinese,1.428223,103.908691,48661,Lettuce with Oyster Sauce 蚝油西生菜,0.0040,Food Delivery,2485343,"[2485215, 2486415)"
2956895,3468960,e176e909ae,w21zu,1729632,2b6fa394,614ff9a3d68f,6,182223,28,1.384277,...,w23bh,chinese,1.428223,103.908691,732277,Wintermelon with Conpoy 原盅冬瓜炖瑶柱,0.0044,Food Delivery,2485343,"[2485215, 2486415)"
2963382,3476589,c8cac4c42d,w21zu,1733291,2b6fa394,951de0fcf9a2,6,183706,28,1.384277,...,w23bh,chinese,1.428223,103.908691,56360,Half,0.0168,Food Delivery,2486226,"[2485215, 2486415)"
2963383,3476590,c8cac4c42d,w21zu,1733291,2b6fa394,5f9f78f5a5a7,6,183706,28,1.384277,...,w23bh,chinese,1.428223,103.908691,25590,Prosperity Smoked Salmon Yu Sheng,0.0152,Food Delivery,2486226,"[2485215, 2486415)"


In [ ]:

#store result in a new csv file
temp.to_csv("20min_different", index=False)

# Finding the avg trip distance of consolidatable orders

In [1]:
# Load Data Using Google Drive URL
import pandas as pd
import numpy as np

def get_gd_csv(gd_id):
	url = f"https://drive.usercontent.google.com/download?id={gd_id}&export=download&authuser=0&confirm=t&uuid=7e92ecad-4c98-448f-8c21-cada2eefe64c&at=AO7h07fO3-BbnWsezAxoi78L-h_0%3A1727109215042"
	df = pd.read_csv(url)
	return df

vendors_gd = '1FPYHV73PfNmC3p8AZEYeyoatJy9XUbAE'
orders_gd = '1ag0k8lzUWQ8J-CopiR4fjN_qv60ErLll'
products_gd = '13V5LUuQB8-VTZX45Eh_UynuQ0ZatAebf'

vendors = get_gd_csv(vendors_gd)
orders = get_gd_csv(orders_gd)
products = get_gd_csv(products_gd)

# Rename columns
vendors.rename(columns={'geohash': 'vendor_geohash', 'latitude': 'vendor_latitude', 'longitude': 'vendor_longitude'}, inplace=True)
orders.rename(columns={'geohash': 'customer_geohash', 'latitude': 'customer_latitude', 'longitude': 'customer_longitude'}, inplace=True)

# Merge orders with vendors and products to get a complete view
merged_data = orders.merge(vendors, on="vendor_id").merge(products, on=["vendor_id", "product_id"])

# Apply the segmentation rule
def segment_delivery(row):
    if 'grocery' in row['name'].lower() or 'combo' in row['name'].lower() or row['primary_cuisine'] == 'grocery':
        return 'Grocery Delivery'
    else:
        return 'Food Delivery'

# Create a new column in the merged dataframe for the segmentation
merged_data['delivery_type'] = merged_data.apply(segment_delivery, axis=1)

# Output the segmented data
food_delivery = merged_data[merged_data['delivery_type'] == 'Food Delivery']
groceries_delivery = merged_data[merged_data['delivery_type'] == 'Grocery Delivery']

# Create a new dataframe to work with
processed = food_delivery.copy()

# Change the format of some columns for easier processing
processed["order_day"] = processed["order_day"].str.replace(' days', '', regex=False)
processed['order_day'] = processed['order_day'].astype(int)
processed["order_time"] = processed["order_time"].str.replace(':', '', regex=False)
processed['order_time'] = processed['order_time'].astype(int)

# Add a column to combine time info from order_time and order_day
def hhmmss_to_seconds(hhmmss):
    hours = hhmmss // 10000
    minutes = (hhmmss % 10000) // 100
    seconds = hhmmss % 100
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return total_seconds

processed['time_delta'] = processed['order_time'].apply(hhmmss_to_seconds) + (24 * 3600 * processed['order_day'])

# Group time info by 20 min ranges
min_value = processed['time_delta'].min()
max_value = processed['time_delta'].max()
bins = range(min_value, max_value + (60 * 20), (60 * 20))

# Create the bins using pd.cut()
processed['time_range'] = pd.cut(processed['time_delta'], bins=bins, right=False)

In [2]:
processed.head()

,Unnamed: 0_x,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day,customer_latitude,...,vendor_geohash,primary_cuisine,vendor_latitude,vendor_longitude,Unnamed: 0,name,unit_price,delivery_type,time_delta,time_range
0,5,f374c8c54c,w21zt,1,21830106,f245bdf79350,4,190303,61,1.340332,...,w21zt,western,1.340332,103.952637,298299,Chicken Cutlet with Rice,0.0020,Food Delivery,5338983,"[5338815, 5340015)"
1,6,f374c8c54c,w21zt,1,21830106,146127be77d4,4,190303,61,1.340332,...,w21zt,western,1.340332,103.952637,890260,Chicken Chop,0.0028,Food Delivery,5338983,"[5338815, 5340015)"
2,7,f374c8c54c,w21zt,1,21830106,6c108c0fb2b9,4,190303,61,1.340332,...,w21zt,western,1.340332,103.952637,692856,Chicken Wings with Rice,0.0016,Food Delivery,5338983,"[5338815, 5340015)"
3,8,f374c8c54c,w21zt,1,21830106,da422c7836e1,4,190303,61,1.340332,...,w21zt,western,1.340332,103.952637,491900,Saba Fish with Rice,0.0024,Food Delivery,5338983,"[5338815, 5340015)"
4,9,f374c8c54c,w21zt,1,21830106,01110c80a0fa,4,190303,61,1.340332,...,w21zt,western,1.340332,103.952637,928344,Chicken Chop with Rice,0.0020,Food Delivery,5338983,"[5338815, 5340015)"


In [21]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

def add_distance_column(df, v_lon_col, v_lat_col, lon_col, lat_col):
    """
    Adds a new column to the DataFrame with the Haversine distance between two sets of coordinates.
    """
    # Calculate distances using the haversine function and assign using .loc for safe operation
    distances = df.apply(lambda row: haversine(row[v_lon_col], row[v_lat_col], row[lon_col], row[lat_col]), axis=1)
    df.loc[:, 'distance_km'] = distances
    return df

processed = add_distance_column(processed, 'vendor_longitude', 'vendor_latitude', 'customer_longitude', 'customer_latitude')
processed
# print(orders_dist)

,Unnamed: 0_x,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day,customer_latitude,...,vendor_latitude,vendor_longitude,Unnamed: 0,name,unit_price,delivery_type,time_delta,time_range,distance_km,trip_id
0,5,f374c8c54c,w21zt,1,21830106,f245bdf79350,4,190303,61,1.340332,...,1.340332,103.952637,298299,Chicken Cutlet with Rice,0.0020,Food Delivery,5338983,"[5338815, 5340015)",0.000000,"21830106.w21ztw21zt[5338815, 5340015)"
1,6,f374c8c54c,w21zt,1,21830106,146127be77d4,4,190303,61,1.340332,...,1.340332,103.952637,890260,Chicken Chop,0.0028,Food Delivery,5338983,"[5338815, 5340015)",0.000000,"21830106.w21ztw21zt[5338815, 5340015)"
2,7,f374c8c54c,w21zt,1,21830106,6c108c0fb2b9,4,190303,61,1.340332,...,1.340332,103.952637,692856,Chicken Wings with Rice,0.0016,Food Delivery,5338983,"[5338815, 5340015)",0.000000,"21830106.w21ztw21zt[5338815, 5340015)"
3,8,f374c8c54c,w21zt,1,21830106,da422c7836e1,4,190303,61,1.340332,...,1.340332,103.952637,491900,Saba Fish with Rice,0.0024,Food Delivery,5338983,"[5338815, 5340015)",0.000000,"21830106.w21ztw21zt[5338815, 5340015)"
4,9,f374c8c54c,w21zt,1,21830106,01110c80a0fa,4,190303,61,1.340332,...,1.340332,103.952637,928344,Chicken Chop with Rice,0.0020,Food Delivery,5338983,"[5338815, 5340015)",0.000000,"21830106.w21ztw21zt[5338815, 5340015)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3431824,4023000,7e3554a009,w21zt,1999998,ff0edd9f,7602ccd87eea,5,220910,55,1.340332,...,1.384277,103.952637,933674,Laksa Prawn Spaghetti,0.0048,Food Delivery,4831750,"[4831215, 4832415)",4.886496,"ff0edd9f.w21zvw21zt[4831215, 4832415)"
3431825,4023001,7e3554a009,w21zt,1999998,ff0edd9f,91129b65ae92,5,220910,55,1.340332,...,1.384277,103.952637,441696,Smoked Duck Penne with Thai Sauce,0.0048,Food Delivery,4831750,"[4831215, 4832415)",4.886496,"ff0edd9f.w21zvw21zt[4831215, 4832415)"
3431826,4023002,7e3554a009,w21zt,1999999,2bba6450,7bb67a5dd524,5,195751,62,1.340332,...,1.384277,103.952637,16318,Tom Yam Fried Rice,0.0028,Food Delivery,5428671,"[5427615, 5428815)",4.886496,"2bba6450.w21zvw21zt[5427615, 5428815)"
3431827,4023003,7e3554a009,w21zt,1999999,2bba6450,d40fe2db522c,5,195751,62,1.340332,...,1.384277,103.952637,727283,Beef,0.0032,Food Delivery,5428671,"[5427615, 5428815)",4.886496,"2bba6450.w21zvw21zt[5427615, 5428815)"


In [22]:
# set a minimum distance of 2.44km (roughly the radius of 1 geohash grid)
processed['distance_km'] = processed['distance_km'].replace(0, 2.44)
processed.sort_values(by='distance_km', ascending=True)


,Unnamed: 0_x,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day,customer_latitude,...,vendor_latitude,vendor_longitude,Unnamed: 0,name,unit_price,delivery_type,time_delta,time_range,distance_km,trip_id
1833456,2147871,7e53438718,w23b4,1055991,6a02f105,dea1ec32fdaa,6,150811,35,1.428223,...,1.428223,103.820801,431620,Bundle for 1,0.0064,Food Delivery,3078491,"[3078015, 3079215)",2.440000,"6a02f105.w23b4w23b4[3078015, 3079215)"
1833437,2147851,de36ab846a,w21zg,1055979,f6750f76,01d7ba03d3f1,0,121849,29,1.384277,...,1.384277,103.864746,652785,Danish Rugbrod,0.0044,Food Delivery,2549929,"[2548815, 2550015)",2.440000,"f6750f76.w21zgw21zg[2548815, 2550015)"
1833436,2147850,de36ab846a,w21zg,1055978,f6750f76,0137484b77d4,3,105456,18,1.384277,...,1.384277,103.864746,295881,Pain au Levain,0.0036,Food Delivery,1594496,"[1593615, 1594815)",2.440000,"f6750f76.w21zgw21zg[1593615, 1594815)"
1833435,2147849,de36ab846a,w21zg,1055977,f6750f76,01d7ba03d3f1,2,103257,10,1.384277,...,1.384277,103.864746,652785,Danish Rugbrod,0.0044,Food Delivery,901977,"[901215, 902415)",2.440000,"f6750f76.w21zgw21zg[901215, 902415)"
1833430,2147843,13e84b4f5b,w21zt,1055974,dbc55e00,337bd21ea708,6,194314,7,1.340332,...,1.340332,103.952637,940347,Aso Ramen + 3 Pcs Gyoza Set,0.0052,Food Delivery,675794,"[675615, 676815)",2.440000,"dbc55e00.w21ztw21zt[675615, 676815)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489491,1742271,0534fbb697,w21xw,849195,2255b645,d7a8745f01de,3,123654,18,1.340332,...,1.296387,103.908691,628547,Mini Tuna Mayo Croissant,0.0016,Food Delivery,1600614,"[1599615, 1600815)",29.715737,"2255b645.w21zkw21xw[1599615, 1600815)"
1489492,1742272,0534fbb697,w21xw,849195,2255b645,b6e5d5cf626d,3,123654,18,1.340332,...,1.296387,103.908691,656413,Mini,0.0008,Food Delivery,1600614,"[1599615, 1600815)",29.715737,"2255b645.w21zkw21xw[1599615, 1600815)"
1489493,1742273,0534fbb697,w21xw,849195,2255b645,73ec7832ebdd,3,123654,18,1.340332,...,1.296387,103.908691,810438,Mini Pain Au Chocolat,0.0008,Food Delivery,1600614,"[1599615, 1600815)",29.715737,"2255b645.w21zkw21xw[1599615, 1600815)"
1489494,1742274,0534fbb697,w21xw,849195,2255b645,bb5a2e76be45,3,123654,18,1.340332,...,1.296387,103.908691,26126,Lemon Tart,0.0180,Food Delivery,1600614,"[1599615, 1600815)",29.715737,"2255b645.w21zkw21xw[1599615, 1600815)"


In [32]:
# Create a new column called trip_id based on vendors from a vendor_geohash sending to a customer_geohash within a specific time_range[AKA CONSOLIDATABLE ORDERS]
processed.loc[:,'trip_id'] = processed['vendor_id'] + '.' + processed['vendor_geohash'] + processed['customer_geohash'] + processed['time_range'].astype(str)
processed

,Unnamed: 0_x,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day,customer_latitude,...,vendor_latitude,vendor_longitude,Unnamed: 0,name,unit_price,delivery_type,time_delta,time_range,distance_km,trip_id
0,5,f374c8c54c,w21zt,1,21830106,f245bdf79350,4,190303,61,1.340332,...,1.340332,103.952637,298299,Chicken Cutlet with Rice,0.0020,Food Delivery,5338983,"[5338815, 5340015)",2.440000,"21830106.w21ztw21zt[5338815, 5340015)"
1,6,f374c8c54c,w21zt,1,21830106,146127be77d4,4,190303,61,1.340332,...,1.340332,103.952637,890260,Chicken Chop,0.0028,Food Delivery,5338983,"[5338815, 5340015)",2.440000,"21830106.w21ztw21zt[5338815, 5340015)"
2,7,f374c8c54c,w21zt,1,21830106,6c108c0fb2b9,4,190303,61,1.340332,...,1.340332,103.952637,692856,Chicken Wings with Rice,0.0016,Food Delivery,5338983,"[5338815, 5340015)",2.440000,"21830106.w21ztw21zt[5338815, 5340015)"
3,8,f374c8c54c,w21zt,1,21830106,da422c7836e1,4,190303,61,1.340332,...,1.340332,103.952637,491900,Saba Fish with Rice,0.0024,Food Delivery,5338983,"[5338815, 5340015)",2.440000,"21830106.w21ztw21zt[5338815, 5340015)"
4,9,f374c8c54c,w21zt,1,21830106,01110c80a0fa,4,190303,61,1.340332,...,1.340332,103.952637,928344,Chicken Chop with Rice,0.0020,Food Delivery,5338983,"[5338815, 5340015)",2.440000,"21830106.w21ztw21zt[5338815, 5340015)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3431824,4023000,7e3554a009,w21zt,1999998,ff0edd9f,7602ccd87eea,5,220910,55,1.340332,...,1.384277,103.952637,933674,Laksa Prawn Spaghetti,0.0048,Food Delivery,4831750,"[4831215, 4832415)",4.886496,"ff0edd9f.w21zvw21zt[4831215, 4832415)"
3431825,4023001,7e3554a009,w21zt,1999998,ff0edd9f,91129b65ae92,5,220910,55,1.340332,...,1.384277,103.952637,441696,Smoked Duck Penne with Thai Sauce,0.0048,Food Delivery,4831750,"[4831215, 4832415)",4.886496,"ff0edd9f.w21zvw21zt[4831215, 4832415)"
3431826,4023002,7e3554a009,w21zt,1999999,2bba6450,7bb67a5dd524,5,195751,62,1.340332,...,1.384277,103.952637,16318,Tom Yam Fried Rice,0.0028,Food Delivery,5428671,"[5427615, 5428815)",4.886496,"2bba6450.w21zvw21zt[5427615, 5428815)"
3431827,4023003,7e3554a009,w21zt,1999999,2bba6450,d40fe2db522c,5,195751,62,1.340332,...,1.384277,103.952637,727283,Beef,0.0032,Food Delivery,5428671,"[5427615, 5428815)",4.886496,"2bba6450.w21zvw21zt[5427615, 5428815)"


In [24]:
# Get the number of unique order_ids, and average distance for each trip_id
unique_counts = processed.groupby('trip_id').agg(unique_orders=('order_id', 'nunique'),dist_per_order=('distance_km', 'mean')).reset_index()
# sort this by the number of unique orders
unique_counts = unique_counts.sort_values('unique_orders', ascending=False)
unique_counts

,trip_id,unique_orders,dist_per_order
369573,"3e0a9276.w21zuw21zu[2320815, 2322015)",13,2.44
775243,"80bcfcd6.w21z8w21z8[67215, 68415)",9,2.44
596447,"6280d003.w23b4w23b4[3427215, 3428415)",9,2.44
763536,"7ef8f3f4.w23b4w23b4[4990815, 4992015)",7,2.44
1492820,"f4232a3c.w21z8w21z8[411615, 412815)",7,2.44
...,...,...,...
541408,"5a12c4b5.w21z7w21z7[5058015, 5059215)",1,2.44
541407,"5a12c4b5.w21z7w21z7[4912815, 4914015)",1,2.44
541406,"5a12c4b5.w21z7w21z7[483615, 484815)",1,2.44
541405,"5a12c4b5.w21z7w21z7[4652415, 4653615)",1,2.44


In [25]:
unique_counts['unique_orders'].mean()

np.float64(1.0718115233567984)

In [29]:
# Remove occurences of only 1 order
unique_counts.drop(unique_counts[unique_counts['unique_orders'] == 1].index, inplace=True)
unique_counts



,trip_id,unique_orders,dist_per_order
369573,"3e0a9276.w21zuw21zu[2320815, 2322015)",13,2.440000
775243,"80bcfcd6.w21z8w21z8[67215, 68415)",9,2.440000
596447,"6280d003.w23b4w23b4[3427215, 3428415)",9,2.440000
763536,"7ef8f3f4.w23b4w23b4[4990815, 4992015)",7,2.440000
1492820,"f4232a3c.w21z8w21z8[411615, 412815)",7,2.440000
...,...,...,...
1552461,"fd9acd2e.w21zuw21zu[5652015, 5653215)",2,2.440000
1496333,"f49aea1a.w21zbw21zb[3002415, 3003615)",2,2.440000
872974,"8fcc5303.w21zew21zg[3676815, 3678015)",2,4.886496
864819,"8e7a3790.w21zbw21zb[2720415, 2721615)",2,2.440000


In [30]:
unique_counts['unique_orders'].mean()

np.float64(2.094746634880949)

In [31]:
unique_counts['dist_per_order'].mean()

np.float64(3.0009189532470315)